This section contains methods on how to reproduce all the results obtained. This is the quickstart tutorial on how to reproduce all results I have obtained

For methods of realignment, see realignment tutorial

All example data used are latest data proAnubis_240716_2142

The Reconstruction algorithm was adapted to the latest data behavior where noise bursts were seen. The data produced in this notebook are designed to run fast, feel free to remove the ignorance on noise bursts and increase the number of chuncks probed to obtain better results.

What is proAnubis_Analaysis_Tools?

proAnubis_Analaysis_Tools is a specialised package for analysing data from the proAnubis setup (https://doi.org/10.22323/1.450.0168) through track reconstruction, time of flight, data quality checks, etc. It works alongside the rawFileReader in Osiris to perform these tasks for TDC event aligned events. 

Within proAnubis_Analyasis_tools are the Reconstructor, ToF tools and general Tools

In [1]:
import importlib
import sys
from tqdm import tqdm
sys.path.insert(1, '..\Osiris Temp\processing\python') # Insert your path to Osiris
import Analysis_tools as ATools
import rawFileReader
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import mplhep as hep
import Timing_tools as TTools
import Reconstruction_tools as RTools
hep.style.use([hep.style.ATLAS])
file_path = '../../Data/proAnubis_240716_2142.raw'

<>:4: SyntaxWarning: invalid escape sequence '\O'
<>:4: SyntaxWarning: invalid escape sequence '\O'
C:\Users\Peter\AppData\Local\Temp\ipykernel_31160\74488883.py:4: SyntaxWarning: invalid escape sequence '\O'
  sys.path.insert(1, '..\Osiris Temp\processing\python') # Insert your path to Osiris


To begin with, These are some useful decorators for certain functions to capture variables if needed. This is better than doing Global variables for clarity and avoid later on mess up certain algorithms. 
Note that if you are unsure what it does, please read https://realpython.com/primer-on-python-decorators/
Please note you need to restart the environment if the function rewrote by the decorator runs into issue, because the function will not return to original state

This is an example on how to get alignment messages, which might come in handy when debugging

In [24]:
#Example Decorators to capture things in the loop so you don't need to redo any calculations later......
def debug_decorator(processedEvents):
    def inner_decorator(func):
        def wrapper(*args, **kwargs):
            result = func(*args, **kwargs)
            insertion_list = result
            update = args[0]
            if insertion_list != [0,0,0,0,0]:
                print(f'New alignment, Event chunk {processedEvents}, insertions {insertion_list}, Updates: {update}')
            return result
        return wrapper
    return inner_decorator

This is an example event loop on how to use fReader Osiris. Please rewrite the file path to your actual raw file that is not on this github due to the size. You need to put the Data file on your own disk and DO NOT try to commit any raw file on the github. becuase raw files are typically large, and synchronization will time out, but the commit will go into history. Then later on if you want to revert, you will have go back and find the exact commit you did, and remove. Which is a pain in the ass



In [25]:
#Jupyter notebook doesn't reload your import even when the content of the file is changed. This is crucial
importlib.reload(rawFileReader)
#The interval is the amount of chunks you want to do realignment analysis with. This also determines the size of the chunk your output is.
interval = 100
fReader = rawFileReader.fileReader(file_path)
# Order is the alignment comparisons you need to specify. for each sublist in order is a pairwise comparison between 2 TDCs.
order = [[0,1], [1,2], [2,3], [3,4]] #Your order to the TDC alignment
# max_process_event_chunk is used to terminate early in the file reading loop. Your total number of events read will be defined as interval * max_proess_event_chunk
max_process_event_chunk = 100
# some initialisation to store things
processedEvents = 0
events = []

#Remember the original function
original_ConstructEventInsertionList = ATools.ConstructEventInsertionList 
# condition to end event loop early
while processedEvents < max_process_event_chunk:
    processedEvents += 1
    #Apply your decorator to change the function
    ATools.ConstructEventInsertionList  = debug_decorator(processedEvents)(original_ConstructEventInsertionList)
    event = fReader.get_aligned_events(order=order, interval=interval)
#reset afterwards
ATools.ConstructEventInsertionList  = original_ConstructEventInsertionList


Or you can do the calculation twice, this might slow things down, but it is more intuitive

In [74]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path) # reload in the classs object
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 100 # End the loop early
processedEvents = 0 # Initialisation

#Initialise variables to store the results
mets = [[] for tdc in range(5)]
empty_header = 0
emtpy_events_with_header = [[], []]
tdc_event_count = [[] for tdc in range(5)]


with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        tdc_event_count_buffer = RTools.find_tdc_event_count(event_chunk)
        [tdc_event_count[i].append(tdc_event_count_buffer[i]) for i in range(5)]
        for idx, (i, j) in enumerate(order):
            x, y, l, m = ATools.find_tdc_alignment_metric(i, j)
            alignMet = ATools.calcAvgAlign(event_chunk, 0, x, y, l, m, i, j, processedEvents)
            empty_header += (alignMet == 100)
            emtpy_events_with_header[0].append(empty_header)
            emtpy_events_with_header[1].append(processedEvents)
            mets[idx].append(alignMet)
        pbar.update(1)
    

Processing Events: 100%|██████████| 100/100 [00:02<00:00, 45.39Events/s]


In [79]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 8))

for idx, item in enumerate(order):
    met = mets[idx]
    i, j = item
    binsx = [x for x in range(len(met))]
    ax.plot(binsx, met, label=f'TDC{i} and TDC{j}, offset 0')


ax.set_xlim(0, max_process_event_chunk)
ax.set_ylim(-1, 40)
ax.legend()
ax.set_title('Alignment graph')
ax.set_ylabel('Average $\sqrt{d\eta^2+d\phi^2}$')
ax.set_xlabel('Processed Event chunks')
plt.show()

<>:15: SyntaxWarning: invalid escape sequence '\s'
<>:15: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Peter\AppData\Local\Temp\ipykernel_13240\4261640906.py:15: SyntaxWarning: invalid escape sequence '\s'
  ax.set_ylabel('Average $\sqrt{d\eta^2+d\phi^2}$')


In [78]:
plt.figure(figsize=(10, 6))
plt.plot(emtpy_events_with_header[1], emtpy_events_with_header[0], marker='o')
plt.xlabel('Processed Events')
plt.ylabel('Only Header')
plt.title('Only Header vs Processed Events')
plt.grid(True)
plt.show()

In [77]:
fig, ax = plt.subplots(figsize=(10, 8))
for tdc in range(5):
    met = tdc_event_count[tdc]
    binsx = [x for x in range(len(met))]
    ax.plot(binsx, met, label=f'TDC{tdc}')

ax.set_xlim(0, max_process_event_chunk)
# ax.set_ylim(-1, 100)
ax.legend()
ax.set_title('TDC number of events recorded')
ax.set_ylabel('num of events')
ax.set_xlabel('Processed Event')
plt.show()
    

In [88]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path) # load in the classs object
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 150 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
TAnalyser = proAnubis_Analysis_Tools.Timing_Analyser(initial_event_chunk,processedEvents)
with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        if event_chunk:
            TAnalyser.update_event(event_chunk, processedEvents)
            status, failure = TAnalyser.check_eta_trigger()
        pbar.update(1)

c:\Users\Peter\OneDrive - University of Cambridge\Desktop\summer2\Documentation\proAnubis_Analysis_Tools.py:316: SyntaxWarning: invalid escape sequence '\e'
  thisHist.cbar.set_label('$\eta$ hit time - $\phi$ hit time, Average (ns)', rotation=270, y=0.3, labelpad=23)
c:\Users\Peter\OneDrive - University of Cambridge\Desktop\summer2\Documentation\proAnubis_Analysis_Tools.py:336: SyntaxWarning: invalid escape sequence '\e'
  thisHist.cbar.set_label('$\eta$ hit time - $\phi$ hit time, Average (ns)', rotation=270, y=0.3, labelpad=23)
Processing Events: 100%|██████████| 150/150 [00:02<00:00, 51.22Events/s]


In [85]:
event_counts_windows = [len(TAnalyser.count[count]) for count in range(7)]
total_windows = sum(event_counts_windows)
normalized_windows = [count / total_windows for count in event_counts_windows]
normalized_windows.append(normalized_windows[-1])
plt.figure(figsize=(12, 8))
r1 = list(range(7)) + [6.5]

# plt.step(r1, normalized_linux, color='mediumseagreen', linestyle='-', linewidth=2, markersize=6, label='0-15000', alpha=1, where='mid')
plt.step(r1, normalized_windows, color='dodgerblue', linestyle='-', linewidth=2, markersize=6, label='0-15000?', alpha=1, where='mid')

plt.xlabel('Trigger Number')
plt.ylabel('Number of Events Normalized')
plt.title('Normalized Event Count')
plt.ylim(0)
plt.xlim(0, 6.5)
plt.xticks(range(7))
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.legend()
plt.show()

In [89]:
TAnalyser.plot_rpc_involvement_histogram()

A reconstruction event loop demonstration for finding the efficiency of each RPCs. Effiiency was calculated using a tag and probe method, where the test RPC is removed, and a track using all other 5 RPCs were fitted. The fitted track is then extrapolated to the test RPC, where the efficiency is probed

In [145]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path) # load in the classs object
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 200 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
tdc_mets = [[] for tdc in range(5)]
Tot_TDC_info = [[] for tdc in range(5)]
with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        event_chunk, tdc_met, TDC_info = fReader.get_aligned_events(order=order, interval=interval, extract_tdc_mets = True)
        [tdc_mets[i].append(tdc_met[i]) for i in range(5) if tdc_met[i] != 0]
        [Tot_TDC_info[i].extend(TDC_info[i]) for i in range(5) if TDC_info[i]]
        pbar.update(1)


Processing Events:   0%|          | 0/200 [00:00<?, ?Events/s]

Processing Events: 100%|██████████| 200/200 [00:03<00:00, 63.83Events/s] 


In [146]:
import matplotlib.pyplot as plt
colors = ['blue', 'green', 'red', 'purple', 'orange']
fig, ax = plt.subplots(figsize=(10, 8))
for tdc in range(5):
    met = tdc_mets[tdc]
    binsx = [x * 25 for x in range(len(met))]
    ax.plot(binsx,met, label = f'tdc {tdc}', color = colors[tdc])

ax.set_xlim(0,max_process_event_chunk)
# ax.set_ylim(0,1)
ax.legend()
ax.set_title('TDC monitoring metric')
ax.set_ylabel('bad time behavior / nominal time behavior')
ax.set_xlabel('processed Event')
plt.show()

In [151]:
importlib.reload(TTools)
TTools.plot_tdc_error_times(Tot_TDC_info)
TTools.plot_tdc_error_times_custom_ranges(Tot_TDC_info, [(0, 100), (100, 200)], output_pdf='Data_output/TDC_first_hit_time.pdf')
TTools.plot_tdc_error_channels(Tot_TDC_info)
TTools.plot_tdc_error_channels_custom_ranges(Tot_TDC_info, [(0, 100), (100, 200)], tdcs_to_plot=None, output_pdf='Data_output/TDC_first_hit_time_channel.pdf')

In [25]:
class Capturer:
    def __init__(self):
        self.TDC_alignment_time = [[] for _ in range(5)]
        self.processedEvents = 0 

    def extra_calculation_decorator(self, func):
        def wrapper(*args, **kwargs):
            minTimes = [300, 300]
            minChans = [-1, -1]
            minRPC = [-1, -1]
            minWord = [-1, -1]
            tdc = [-1, -1]
            eta = kwargs.get('eta', True)
            
            rpc1Hits = args[0]
            rpc2Hits = args[1]
            skipChans = kwargs.get('skipChans', [])

            # Perform the core function
            result = func(*args, **kwargs)
            
            if result == -1:
                return result
            print(rpc1Hits)
            print(rpc2Hits)
            for hit in rpc1Hits:
                if hit.time < minTimes[0] and hit.channel not in skipChans:
                    minTimes[0] = hit.time
                    minChans[0] = hit.channel
                    minRPC[0] = hit.rpc
            for hit in rpc2Hits:
                if hit.time < minTimes[1] and hit.channel not in skipChans:
                    minTimes[1] = hit.time
                    minChans[1] = hit.channel
                    minRPC[1] = hit.rpc
            
            if -1 in minChans:
                return -1
            
            # Assuming rpcHitToTdcChan is a function available in the context
            a = TTools.rpcHitToTdcChan(minRPC[0], minChans[0], eta)
            b = TTools.rpcHitToTdcChan(minRPC[1], minChans[1], eta)
            
            tdc[0], minWord[0] = a
            tdc[1], minWord[1] = b

            # Update the TDC_alignment_time attribute
            self.TDC_alignment_time[tdc[0]].append((minTimes[0], a, self.processedEvents))
            self.TDC_alignment_time[tdc[1]].append((minTimes[1], b, self.processedEvents))

            return result

        return wrapper

In [18]:
original_testAlign = ATools.testAlign

In [22]:
ATools.testAlign = original_testAlign

In [26]:
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)

Capturer = Capturer()

# Apply the decorator
original_testAlign = ATools.testAlign
ATools.testAlign = Capturer.extra_calculation_decorator(ATools.testAlign)

# Main loop
interval = 100  # Set your monitoring chunk size
fReader = rawFileReader.fileReader(file_path)  # Load in the class object
order = [[0, 1], [1, 2], [2, 3], [3, 4]]  # Order what you want to align
max_process_event_chunk = 1000  # End the loop early
processedEvents = 0  # Initialization
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
tdc_mets = [[] for tdc in range(5)]
Tot_TDC_info = [[] for tdc in range(5)]

with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        pbar.update(1)

# Restore the original function
ATools.testAlign = original_testAlign

# Process or print the TDC_alignment_time as needed
# print(tdc_manager.TDC_alignment_time)

[<Analysis_tools.rpcHit object at 0x000001DB32B17530>, <Analysis_tools.rpcHit object at 0x000001DB32B17590>, <Analysis_tools.rpcHit object at 0x000001DB32B16F90>, <Analysis_tools.rpcHit object at 0x000001DB32B15580>]


TypeError: cannot unpack non-iterable NoneType object

In [16]:
TTools.plot_tdc_alignment_channels_separate_ranges(tdc_manager.TDC_alignment_time)

TypeError: 'NoneType' object is not subscriptable

In [32]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path) # load in the classs object
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 100 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
TAnalyser = proAnubis_Analysis_Tools.Timing_Analyser(initial_event_chunk,processedEvents)
while processedEvents < max_process_event_chunk:
    processedEvents += 1
    event_chunk = fReader.get_aligned_events(order=order, interval=interval)
    if event_chunk:
        TAnalyser.update_event(event_chunk, processedEvents)
        TAnalyser.readTDCTimeDiffs()
        
outDict = {'totDiffs':TAnalyser.totDiffs,
                    'nDiffs':TAnalyser.nDiffs,
                    'diffHists':TAnalyser.diffHists} 

In [33]:
residEta, residPhi = TAnalyser.Calculate_Residual_and_plot_TDC_Time_Diffs(outDict, 
                                                     pdf_filename='Data_output/TDC_time_diffs.pdf', 
                                                     max_itr = 5)

In [34]:
import matplotlib.pyplot as plt
rpcNames = {0:"Triplet Low",1: "Triplet Mid", 2:"Triplet Top", 3:"Singlet",4:"Doublet Low",5:"Doublet Top"}
fig, ax = plt.subplots(1, figsize=(16, 8), dpi=100)
phichannels = [x-0.5 for x in range(65)]

for idx, rpc in enumerate([0,1,2,3,4,5]):
    plotPhiResids = residPhi[idx].copy()
    plotPhiResids.append(plotPhiResids[-1])
    plt.step(phichannels,plotPhiResids,linewidth=3,label=rpcNames[rpc],where='post')
yrange = ax.get_ylim()
#ax.text(40, 0.8*yrange[1], "Slope: "+str(round(popt[0],3))+" ns/channel, Offset: "+str(round(popt[1],2))+" ns", fontsize=14,
#               verticalalignment='top')
ax.set_xlabel('$\phi$ Channel')
ax.set_ylabel('Time Residual (ns)')
# ax.set_ylim([-6,6])
ax.set_xlim([-0.5,63.5])
plt.legend()
plt.show()
fig, ax = plt.subplots(1, figsize=(16, 8), dpi=100)
etchannels = [x-0.5 for x in range(33)]
for idx, rpc in enumerate([0,1,2,3,4,5]):
    plotEtaResids = residEta[idx].copy()
    plotEtaResids.append(plotEtaResids[-1])
    plt.step(etchannels,plotEtaResids,linewidth=3,label=rpcNames[rpc],where='post')
yrange = ax.get_ylim()
#ax.text(40, 0.8*yrange[1], "Slope: "+str(round(popt[0],3))+" ns/channel, Offset: "+str(round(popt[1],2))+" ns", fontsize=14,
#               verticalalignment='top')
ax.set_xlabel('$\eta$ Channel')
ax.set_ylabel('Time Residual (ns)')
# ax.set_ylim([-6,6])
ax.set_xlim([-0.5,31.5])
plt.legend()
plt.show()     

<>:13: SyntaxWarning: invalid escape sequence '\p'
<>:28: SyntaxWarning: invalid escape sequence '\e'
<>:13: SyntaxWarning: invalid escape sequence '\p'
<>:28: SyntaxWarning: invalid escape sequence '\e'
C:\Users\Peter\AppData\Local\Temp\ipykernel_13240\2682300778.py:13: SyntaxWarning: invalid escape sequence '\p'
  ax.set_xlabel('$\phi$ Channel')
C:\Users\Peter\AppData\Local\Temp\ipykernel_13240\2682300778.py:28: SyntaxWarning: invalid escape sequence '\e'
  ax.set_xlabel('$\eta$ Channel')


In [ ]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path) # load in the classs object
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 100 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
reconstructor = proAnubis_Analysis_Tools.Reconstructor(initial_event_chunk, processedEvents)
with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        #Zone of Reconstruction
        if event_chunk:
            reconstructor.update_event(event_chunk, processedEvents)
            reconstructor.populate_hits()
            reconstructor.apply_systematic_correction(residEta, residPhi)
            cluster = reconstructor.make_cluster()
            reconstructor.reconstruct_and_extrapolate(cluster)
        pbar.update(1)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
for RPC in range(6):
    if reconstructor.possible_reconstructions[RPC] == 0:
        efficiency = [0 for x in reconstructor.successful_reconstructions[RPC]]
    else:
        efficiency = [x / reconstructor.possible_reconstructions[RPC] for x in reconstructor.successful_reconstructions[RPC]]
    plt.plot(reconstructor.tol, efficiency, label=f'RPC {RPC}')

plt.xlabel('Tolerance')
plt.ylabel('Efficiency')
plt.title('idc what the title is')
plt.legend()
plt.grid(True)
plt.show()
print(reconstructor.possible_reconstructions)

In [ ]:
import matplotlib.colors as colors
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
rpcNames = {0: "Triplet Low", 1: "Triplet Mid", 2: "Triplet Top", 3: "Singlet", 4: "Doublet Low", 5: "Doublet Top"}

success_events = [[0 for etchan in range(32)] for phchan in range(64)]

with PdfPages('Data_output/reconstruction_heatmap_plots.pdf') as pdf:
    for rpc in range(6):
        for ph in range(64):
            for et in range(32):
                if reconstructor.successful_reconstructed_coords[rpc][ph][et] > 0:
                    total_successful = reconstructor.successful_reconstructed_coords[rpc][ph][et]
                    total_events = reconstructor.possible_reconstructions_coords[rpc][ph][et]
                    if total_events > 0:
                        success_events[ph][et] = total_successful / total_events
                    else:
                        success_events[ph][et] = 0  # No events, efficiency is 0

        fig, ax = plt.subplots(1, figsize=(16, 8), dpi=100)
        etachannels = [x - 0.5 for x in range(33)]
        phichannels = [x - 0.5 for x in range(65)]
        etaHist = (success_events, np.array(phichannels), np.array(etachannels))
        zrange = [0, max(max(row) for row in success_events)]
        thisHist = hep.hist2dplot(etaHist, norm=colors.Normalize(zrange[0], zrange[1]))
        thisHist.cbar.set_label('Successful reconstructions / Possible reconstructions', rotation=270, y=0.3, labelpad=23)
        plt.ylim(31.5, -0.5)
        plt.ylabel("Eta Channel")
        plt.xlabel("Phi Channel")
        ax.set_title(rpcNames[rpc])

        # Draw lines
        x_points = [-0.5, 64.5]
        y_points = [7.5, 15.5, 23.5]
        for y_point in y_points:
            plt.plot(x_points, [y_point, y_point], 'k', linestyle='dotted')
        y_points = [-0.5, 31.5]
        x_points = [7.5, 15.5, 23.5, 31.5, 39.5, 47.5, 55.5]
        for x_point in x_points:
            plt.plot([x_point, x_point], y_points, 'k', linestyle='dashed')

        pdf.savefig(fig)
        plt.close(fig)

print("PDF created successfully.")

Combining results from timing analysis to the reconstructor

In [80]:
importlib.reload(rawFileReader) # Reload fReader
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(ATools)
interval = 100 # Set your monitoring chunck size
fReader = rawFileReader.fileReader(file_path) # load in the classs object
order = [[0,1], [1,2], [2,3], [3,4]] # Order what you want to align
max_process_event_chunk = 150 # End the loop early
processedEvents = 0 # Initialisation
initial_event_chunk = fReader.get_aligned_events(order=order, interval=interval)
reconstructor = proAnubis_Analysis_Tools.Reconstructor(initial_event_chunk, processedEvents, tof_correction=True)
TAnalyser = proAnubis_Analysis_Tools.Timing_Analyser(initial_event_chunk,processedEvents)
with tqdm(total=max_process_event_chunk, desc="Processing Events", unit='Events') as pbar:
    while processedEvents < max_process_event_chunk:
        processedEvents += 1
        event_chunk = fReader.get_aligned_events(order=order, interval=interval)
        if event_chunk:
            reconstructor.update_event(event_chunk, processedEvents)
            # if processedEvents < 250:
            #     pbar.update(1)
            #     continue
            reconstructor.populate_hits()
            reconstructor.apply_systematic_correction(residEta, residPhi)
            cluster = reconstructor.make_cluster()
            filtered_events = RTools.filter_events(cluster,1,6)     
            reconstructor.extract_angles_phi_eta_timed_DZ_modified(filtered_events)
        pbar.update(1)

c:\Users\Peter\OneDrive - University of Cambridge\Desktop\summer2\Documentation\proAnubis_Analysis_Tools.py:316: SyntaxWarning: invalid escape sequence '\e'
  for y_point in y_points_dotted:
c:\Users\Peter\OneDrive - University of Cambridge\Desktop\summer2\Documentation\proAnubis_Analysis_Tools.py:336: SyntaxWarning: invalid escape sequence '\e'
  plt.plot([x_point, x_point], y_points_dashed, 'k', linestyle='dashed')
Processing Events: 100%|██████████| 150/150 [00:19<00:00,  7.62Events/s]


In [81]:
import matplotlib.pyplot as plt
import numpy as np
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(14, 20))
bin_edges = np.arange(-90.5, 91.5, 1)
phi_edges = np.arange(-180.5, 181.5, 1)
ax1.bar(bin_edges[:-1], reconstructor.eta_histogram, width=1, edgecolor='black', align='edge')
ax1.set_title('eta Angles Histogram chunk3')
ax1.set_xlabel('eta Angle (degrees)')
ax1.set_ylabel('Counts')

ax2.bar(bin_edges[:-1], reconstructor.phi_histogram, width=1, edgecolor='black', align='edge')
ax2.set_title('phi Angles Histogram chunk3')
ax2.set_xlabel('phi Angle (degrees)')
ax2.set_ylabel('Counts')

ax3.bar(phi_edges[:-1], reconstructor.solid_theta_histogram, width=1, edgecolor='black', align='edge')
ax3.set_title('solid theta Angles Histogram chunk3')
ax3.set_xlabel('solid theta Angle (degrees)')
ax3.set_ylabel('Counts')

ax4.bar(phi_edges[:-1], reconstructor.solid_phi_histogram, width=1, edgecolor='black', align='edge')
ax4.set_title('solid phi Angles Histogram chunk3')
ax4.set_xlabel('solid phi Angle (degrees)')
ax4.set_ylabel('Counts')


plt.tight_layout()
plt.show()